# Prerrequisites

Installing Spark and Apache Kafka Library in VM


---



In [1]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz

# unzip it
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install py4j

# For maps
!pip install folium
!pip install plotly

     |████████████████████████████████| 204kB 12.4MB/s 


Define the environment (Java & Spark homes)

---

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Starting Spark Session and print the version


---


In [3]:
import findspark
findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.ui.port", "4500") \
        .getOrCreate()

spark.version

'3.0.1'

In [4]:
spark

In [5]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

Creating ngrok tunnel to allow Spark UI (Optional)


In [6]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
!sleep 2
get_ipython().system_raw('./ngrok http 4500 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-12-10 18:17:19--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.169.225.215, 54.198.148.204, 52.20.200.43, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.169.225.215|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  13.9MB/s    in 0.9s    

2020-12-10 18:17:21 (13.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


# Descargar Datasets

In [7]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/bank.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/vehicles.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/characters.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/netflix_titles.csv -P /dataset
!ls /dataset

bank.csv  characters.csv  netflix_titles.csv  vehicles.csv


# Lectura de Datos con Spark SQL

---



## Ejemplo 1

In [8]:
!head /dataset/bank.csv

"age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"
30;"unemployed";"married";"primary";"no";1787;"no";"no";"cellular";19;"oct";79;1;-1;0;"unknown";"no"
33;"services";"married";"secondary";"no";4789;"yes";"yes";"cellular";11;"may";220;1;339;4;"failure";"no"
35;"management";"single";"tertiary";"no";1350;"yes";"no";"cellular";16;"apr";185;1;330;1;"failure";"no"
30;"management";"married";"tertiary";"no";1476;"yes";"yes";"unknown";3;"jun";199;4;-1;0;"unknown";"no"
59;"blue-collar";"married";"secondary";"no";0;"yes";"no";"unknown";5;"may";226;1;-1;0;"unknown";"no"
35;"management";"single";"tertiary";"no";747;"no";"no";"cellular";23;"feb";141;2;176;3;"failure";"no"
36;"self-employed";"married";"tertiary";"no";307;"yes";"no";"cellular";14;"may";341;1;330;2;"other";"no"
39;"technician";"married";"secondary";"no";147;"yes";"no";"cellular";6;"may";151;2;-1;0;"unknown";"no"
41;"entrepreneur";"marrie

Leyendo Datos desde un RDD, y conviertiéndolo a DataFrame

In [9]:
from pyspark.sql.types import Row
from pyspark.sql.functions import *

bankText = spark.sparkContext.textFile("/dataset/bank.csv")

bank = bankText.map(lambda lineaCsv: lineaCsv.split(";"))\
.filter(lambda s: s[0] != "\"age\"") \
.map(lambda row: Row(int(row[0]), row[1].replace("\"", ""), row[2].replace("\"", ""), row[3].replace("\"", ""), row[5].replace("\"", ""))) \
.toDF(["age", "job", "marital", "education", "balance"]) \
.withColumn("age", col("age").cast("int"))

In [10]:
bank.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- balance: string (nullable = true)



In [11]:
bank.registerTempTable("bank")

Cargamos una **extensión de Google Colab** para mostrar una tabla con posibilidad de filtro (Debemos devolver un DataFrame en Pandas)

In [12]:
%load_ext google.colab.data_table

In [13]:

from pyspark.sql.functions import *

bank_grouped = bank\
.groupBy(bank.marital) \
.agg({"balance": "avg"}) \
.select("marital", col("avg(balance)").alias("balance_avg")) \
.orderBy(col("balance_avg").desc())\

bank_grouped.show()


+--------+------------------+
| marital|       balance_avg|
+--------+------------------+
| married| 1463.195566678584|
|  single|1460.4147157190635|
|divorced|1122.3901515151515|
+--------+------------------+



In [ ]:
# Here, we group all the data splited in the worker nodes in the driver. WE LOSE PARALELISME
bank_grouped.toPandas()

spark-3.0.1-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:88: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.15.1 must be installed; however, your version was 0.14.1.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,marital,balance_avg
0,married,1463.195567
1,single,1460.414716
2,divorced,1122.390152


In [ ]:
spark.sql("SELECT marital, avg(balance) as balance_avg FROM bank group by marital").show()

+--------+------------------+
| marital|       balance_avg|
+--------+------------------+
|divorced|1122.3901515151515|
| married| 1463.195566678584|
|  single|1460.4147157190635|
+--------+------------------+



In [ ]:
import plotly.express as px

fig = px.pie(bank_grouped.toPandas(), values='balance_avg', names='marital', title='By Marital')
fig.show()

spark-3.0.1-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:88: UserWarning:

toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.15.1 must be installed; however, your version was 0.14.1.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.



## Ejemplo 2

Cargando un fichero CSV como RDD y convirtiendo a DataFrame aplicando un esquema específico usando el método **"createDataFrame"**

In [ ]:
from pyspark.sql.types import *

bankSchema = StructType([
    StructField("age", IntegerType(), False), 
    StructField("job", StringType(), False),
    StructField("marital", StringType(), False),
    StructField("education", StringType(), False),
    StructField("balance", IntegerType(), False)])

bankText = spark.sparkContext.textFile("/dataset/bank.csv")

bank = bankText\
.map(lambda s: s.split(";")).filter(lambda s: s[0] != "\"age\"")\
.map(lambda s:(int(s[0]), str(s[1]).replace("\"", ""), str(s[2]).replace("\"", ""), str(s[3]).replace("\"", ""), int(s[5]) ))

bankdf = spark.createDataFrame(bank, bankSchema)
bankdf.registerTempTable("bank2")

In [ ]:
spark.sql("select * from bank2 limit 10").show()

+---+-------------+-------+---------+-------+
|age|          job|marital|education|balance|
+---+-------------+-------+---------+-------+
| 30|   unemployed|married|  primary|   1787|
| 33|     services|married|secondary|   4789|
| 35|   management| single| tertiary|   1350|
| 30|   management|married| tertiary|   1476|
| 59|  blue-collar|married|secondary|      0|
| 35|   management| single| tertiary|    747|
| 36|self-employed|married| tertiary|    307|
| 39|   technician|married|secondary|    147|
| 41| entrepreneur|married| tertiary|    221|
| 43|     services|married|  primary|    -88|
+---+-------------+-------+---------+-------+



## Ejercicio 1
**Carga el fichero "vehicles.csv" directamente en un DataFrame, muéstra el contenido por pantalla e imprime el esquema**

Apóyate en la siguiente documentación para hacer la lectura directamente en un DataFrame https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html

---



In [14]:
!head /dataset/vehicles.csv

name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class
Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled
T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft
X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,NA,repulsorcraft
TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,NA,6.4,1200,1,0,65,2 days,starfighter
Snowspeeder,t-47 airspeeder,Incom corporation,NA,4.5,650,2,0,10,none,airspeeder
TIE bomber,TIE/sa bomber,Sienar Fleet Systems,NA,7.8,850,1,0,none,2 days,space/planetary bomber
AT-AT,All Terrain Armored Transport,"Kuat Drive Yards, Imperial Department of Military Research",NA,20,60,5,40,1000,NA,assault walker
AT-ST,All Terrain Scout Transport,"Kuat Drive Yards, Imperial Department of Military Research",NA,2,90,2,0,200,none,walker
Storm IV Twin-Pod cloud car,Storm IV Twin-Pod,Bespin

In [15]:
# Load the vehicles.csv
vehicles = spark.read.load("/dataset/vehicles.csv", 
                           format="csv", sep=",", header=True, inferSchema=True)

In [16]:
# Show the content
vehicles.show()

+--------------------+--------------------+--------------------+---------------+------+----------------------+----+----------+--------------+---------------+--------------------+
|                name|               model|        manufacturer|cost_in_credits|length|max_atmosphering_speed|crew|passengers|cargo_capacity|    consumables|       vehicle_class|
+--------------------+--------------------+--------------------+---------------+------+----------------------+----+----------+--------------+---------------+--------------------+
|        Sand Crawler|      Digger Crawler|Corellia Mining C...|         150000|  36.8|                    30|  46|        30|         50000|       2 months|             wheeled|
|      T-16 skyhopper|      T-16 skyhopper|   Incom Corporation|          14500|  10.4|                  1200|   1|         1|            50|              0|       repulsorcraft|
|    X-34 landspeeder|    X-34 landspeeder|SoroSuub Corporation|          10550|   3.4|                  

In [17]:
# Print schema
vehicles.printSchema()

root
 |-- name: string (nullable = true)
 |-- model: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- cost_in_credits: string (nullable = true)
 |-- length: string (nullable = true)
 |-- max_atmosphering_speed: string (nullable = true)
 |-- crew: integer (nullable = true)
 |-- passengers: string (nullable = true)
 |-- cargo_capacity: string (nullable = true)
 |-- consumables: string (nullable = true)
 |-- vehicle_class: string (nullable = true)



In [18]:
# We cast the column cargo_capacity to an integer in a new dataframe (remember: we cannot change a RDD)
new_vehicles = vehicles. \
withColumn("cargo_capacity", col("cargo_capacity").cast("int"))

In [19]:
# Print schema
new_vehicles.printSchema()

root
 |-- name: string (nullable = true)
 |-- model: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- cost_in_credits: string (nullable = true)
 |-- length: string (nullable = true)
 |-- max_atmosphering_speed: string (nullable = true)
 |-- crew: integer (nullable = true)
 |-- passengers: string (nullable = true)
 |-- cargo_capacity: integer (nullable = true)
 |-- consumables: string (nullable = true)
 |-- vehicle_class: string (nullable = true)



**Filtra el anterior dataframe obteniendo los vehículos cuya capacidad supere 70**


---



In [20]:
new_vehicles \
.filter(vehicles.cargo_capacity > 70) \
.show()

+--------------------+--------------------+--------------------+---------------+------+----------------------+----+----------+--------------+---------------+--------------------+
|                name|               model|        manufacturer|cost_in_credits|length|max_atmosphering_speed|crew|passengers|cargo_capacity|    consumables|       vehicle_class|
+--------------------+--------------------+--------------------+---------------+------+----------------------+----+----------+--------------+---------------+--------------------+
|        Sand Crawler|      Digger Crawler|Corellia Mining C...|         150000|  36.8|                    30|  46|        30|         50000|       2 months|             wheeled|
|               AT-AT|All Terrain Armor...|Kuat Drive Yards,...|             NA|    20|                    60|   5|        40|          1000|             NA|      assault walker|
|               AT-ST|All Terrain Scout...|Kuat Drive Yards,...|             NA|     2|                  

# Spark SQL. Funciones de agregación

Links útiles:

https://spark.apache.org/docs/latest/api/python/pyspark.sql.html


## Ejercicio 2

**A partir del dataframe con todos los vehículos cargado en el ejercicio 1, obtén el número de pasarejos medios por clase de vehículo**


---




In [21]:
# First, we cast the passengers into an integer (alternative way)
from pyspark.sql.types import IntegerType

vehicles3 = new_vehicles \
.withColumn("passengers", col("passengers").cast(IntegerType()))

vehicles3.printSchema()

root
 |-- name: string (nullable = true)
 |-- model: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- cost_in_credits: string (nullable = true)
 |-- length: string (nullable = true)
 |-- max_atmosphering_speed: string (nullable = true)
 |-- crew: integer (nullable = true)
 |-- passengers: integer (nullable = true)
 |-- cargo_capacity: integer (nullable = true)
 |-- consumables: string (nullable = true)
 |-- vehicle_class: string (nullable = true)



In [22]:
# Now, we calculate the average
vehicles3 \
.groupBy("vehicle_class") \
.avg("passengers") \
.show()

+--------------------+------------------+
|       vehicle_class|   avg(passengers)|
+--------------------+------------------+
|          droid tank|               4.0|
|space/planetary b...|               0.0|
|   droid starfighter|               0.0|
|      wheeled walker|             150.5|
|          sail barge|             500.0|
|       landing craft|             284.0|
|             speeder|0.3333333333333333|
|      assault walker|              40.0|
|         starfighter|               0.0|
|         air speeder|               0.0|
|           transport|               6.0|
|repulsorcraft car...|              16.0|
|             wheeled|              30.0|
|          airspeeder|               0.8|
|           submarine|               2.0|
|             gunship|              15.0|
|       repulsorcraft|17.285714285714285|
|fire suppression ...|              null|
|              walker|              16.5|
+--------------------+------------------+



In [ ]:
# Another way (in case we want to calculate one than more field)
vehicles3 \
.groupBy("vehicle_class") \
.agg(avg("passengers"), avg("crew")) \
.show()

+--------------------+------------------+---------+
|       vehicle_class|   avg(passengers)|avg(crew)|
+--------------------+------------------+---------+
|          droid tank|               4.0|      0.0|
|space/planetary b...|               0.0|      1.0|
|   droid starfighter|               0.0|      1.0|
|      wheeled walker|             150.5|     10.5|
|          sail barge|             500.0|     26.0|
|       landing craft|             284.0|    140.0|
|             speeder|0.3333333333333333|      1.0|
|      assault walker|              40.0|      5.0|
|         starfighter|               0.0|     0.75|
|         air speeder|               0.0|      2.0|
|           transport|               6.0|      2.0|
|repulsorcraft car...|              16.0|      5.0|
|             wheeled|              30.0|     46.0|
|          airspeeder|               0.8|      1.2|
|           submarine|               2.0|      1.0|
|             gunship|              15.0|      3.5|
|       repu

## Ejercicio 3

**Carga el fichero "characters.csv" y obtén el color de ojos más común entre todos los personajes**

---

In [ ]:
# Load the characters.csv
characters = spark.read.load("/dataset/characters.csv", 
                           format="csv", sep=",", header=True, inferSchema=True)

In [ ]:
# Print the schema
characters.printSchema()

root
 |-- name: string (nullable = true)
 |-- height: string (nullable = true)
 |-- mass: string (nullable = true)
 |-- hair_color: string (nullable = true)
 |-- skin_color: string (nullable = true)
 |-- eye_color: string (nullable = true)
 |-- birth_year: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- homeworld: string (nullable = true)
 |-- species: string (nullable = true)



In [ ]:
characters.show()

+--------------------+------+-----+-------------+----------------+---------+----------+-------------+----------+--------------+
|                name|height| mass|   hair_color|      skin_color|eye_color|birth_year|       gender| homeworld|       species|
+--------------------+------+-----+-------------+----------------+---------+----------+-------------+----------+--------------+
|      Luke Skywalker|   172|   77|        blond|            fair|     blue|     19BBY|         male|  Tatooine|         Human|
|               C-3PO|   167|   75|           NA|            gold|   yellow|    112BBY|           NA|  Tatooine|         Droid|
|               R2-D2|    96|   32|           NA|     white, blue|      red|     33BBY|           NA|     Naboo|         Droid|
|         Darth Vader|   202|  136|         none|           white|   yellow|   41.9BBY|         male|  Tatooine|         Human|
|         Leia Organa|   150|   49|        brown|           light|    brown|     19BBY|       female|  A

In [ ]:
characters \
.groupBy("eye_color") \
.agg(count("eye_color")) \
.orderBy("count(eye_color)", ascending=False) \
.limit(1) \
.show()

+---------+----------------+
|eye_color|count(eye_color)|
+---------+----------------+
|    brown|              21|
+---------+----------------+



In [ ]:
# Another way
characters \
.groupBy("eye_color") \
.count() \
.orderBy(col("count").desc()) \
.limit(1) \
.show()

+---------+-----+
|eye_color|count|
+---------+-----+
|    brown|   21|
+---------+-----+



## Ejercicio 4

**Carga el dataframe completo de personajes en una tabla temporal y obtén mediante SQL el número de personajes por género**


---



In [ ]:
characters.registerTempTable("characters_table")

spark.sql("select gender, count(1) as total from characters_table group by gender order by total desc").show()

+-------------+-----+
|       gender|total|
+-------------+-----+
|         male|   62|
|       female|   19|
|           NA|    3|
|         none|    2|
|hermaphrodite|    1|
+-------------+-----+



In [ ]:
# Group NA and none values
characters2 = characters \
.withColumn("gender", expr("case when gender = 'NA' then 'none' else gender end")) \

In [ ]:
characters2.registerTempTable("characters2_table")

spark.sql("select gender, count(1) as total from characters2_table group by gender order by total desc").show()

+-------------+-----+
|       gender|total|
+-------------+-----+
|         male|   62|
|       female|   19|
|         none|    5|
|hermaphrodite|    1|
+-------------+-----+



## Ejercicio 5

**Carga el fichero "netflix_titles.csv" en un DataFrame e imprime el esquema**


---



In [23]:
!head /dataset/netflix_titles.csv

show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole Howard, Jennifer Cameron, Jonathan Holmes, Lee Tockar, Lisa Durupt, Maya Kay, Michael Dobson","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies","Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first."
80117401,Movie,Jandino: Whatever it Takes,,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,"Jandino Asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of ""Sex on Fire"" in his comedy show."
70234439,TV Show,Transformers Prime,,"Peter Cullen, Sumalee Montano, Frank Welker, Jeffrey Combs, Kevin Michael Richardson, Tania Gun

In [24]:
# Load csv
netflix_titles = spark.read.load("/dataset/netflix_titles.csv", 
                                 format="csv", sep=",", inferSchema=True, header=True)

In [25]:
# Print schema
netflix_titles.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



## Ejercicio 6

**Obtén el año en el que se añadieron más películas (No TV Shows)**
**Usa una UDF para obtener el año en el que se añadió al catálogo**

---



In [38]:
from pyspark.sql.functions import *

# UDF to extract year of date_added
def getYearAdded(s): return s[-4:]
# Register UDF in Spark
getYearAdded_udf = udf(getYearAdded)

netflix_titles.filter(netflix_titles["type"] == "Movie") \
              .filter(col("date_added").isNotNull()) \
              .select(getYearAdded_udf("date_added").alias("year_added")) \
              .groupBy("year_added") \
              .count() \
              .show()

+----------+-----+
|year_added|count|
+----------+-----+
|      2016|  264|
|      2012|    4|
|      2020|  146|
|      2019| 1545|
|      2017|  911|
|      2014|   19|
|      2013|    6|
|      2018| 1288|
|      2009|    2|
|      2011|   13|
|      2008|    1|
|      2015|   57|
|      2010|    1|
+----------+-----+

